In [1]:
import pandas as pd


In [6]:
with open('BPI2016_Clicks_NOT_Logged_In.csv', 'r', encoding='utf-8') as f:
    for _ in range(5):
        line = f.readline()
        print(line.strip(';'))
        

SessionID;IPID;TIMESTAMP;VHOST;URL_FILE;PAGE_NAME;REF_URL_category;page_load_error;page_action_detail;tip;service_detail;xps_info;page_action_detail_EN;service_detail_EN;tip_EN

9709637;32252;2015-09-09 08:41:16.933000000;www.werk.nl;/werk_nl/werkgever/cvs_zoeken;cvs_zoeken;;0;;;;;;;

24410332;328776;2015-09-09 11:17:33.703000000;www.werk.nl;/werk_nl/werknemer/home;home;;0;;;;;;;

12989875;806282;2015-09-08 18:32:30.523000000;digid.werk.nl;/portal/page/portal/home/diensten/aanvragen-ww;aanvragen-ww;;0;;;2. Uw laatste werkgever;;;2. Your last employer;

4775769;1374967;2015-09-08 12:08:05.763000000;digid.werk.nl;/portal/page/portal/home/diensten/aanvragen-tw;aanvragen-tw;;0;;;;;;;



In [12]:
df = pd.read_csv('BPI2016_Clicks_NOT_Logged_In.csv', nrows=150, delimiter=';')
df

,SessionID,IPID,TIMESTAMP,VHOST,URL_FILE,PAGE_NAME,REF_URL_category,page_load_error,page_action_detail,tip,service_detail,xps_info,page_action_detail_EN,service_detail_EN,tip_EN
0,9709637,32252,2015-09-09 08:41:16.933000000,www.werk.nl,/werk_nl/werkgever/cvs_zoeken,cvs_zoeken,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24410332,328776,2015-09-09 11:17:33.703000000,www.werk.nl,/werk_nl/werknemer/home,home,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12989875,806282,2015-09-08 18:32:30.523000000,digid.werk.nl,/portal/page/portal/home/diensten/aanvragen-ww,aanvragen-ww,NaN,0,NaN,NaN,2. Uw laatste werkgever,NaN,NaN,2. Your last employer,NaN
3,4775769,1374967,2015-09-08 12:08:05.763000000,digid.werk.nl,/portal/page/portal/home/diensten/aanvragen-tw,aanvragen-tw,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27370352,805833,2015-09-09 10:41:02.817000000,www.werk.nl,/werk_nl/werknemer/uitkering-aanvragen/ww,ww,UWV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,638770,1571348,2015-09-09 11:57:04.893000000,www.werk.nl,/werk_nl/werkgever/cvs_zoeken,cvs_zoeken,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,26805371,1383303,2015-09-08 12:09:51.040000000,www.werk.nl,/werk_nl/werknemer/werkmap,werkmap,UWV,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,13748886,1512386,2015-09-08 21:39:10.950000000,www.werk.nl,/werk_nl/werknemer/vacatures,vacatures,Search Engine,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,28911732,237498,2015-09-09 13:00:39.443000000,www.werk.nl,/werk_nl/werkgever/cvs_zoeken,cvs_zoeken,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.columns

Index(['SessionID', 'IPID', 'TIMESTAMP', 'VHOST', 'URL_FILE', 'PAGE_NAME',
       'REF_URL_category', 'page_load_error', 'page_action_detail', 'tip',
       'service_detail', 'xps_info', 'page_action_detail_EN',
       'service_detail_EN', 'tip_EN'],
      dtype='object')

In [14]:
df.to_csv('temp.csv',index=False)